# Assignment 1 - Player Analysis

Analyze players of varying abilities using basic and advanced metrics. Think like an analyst — what do the stats really say?

Analyze a single season for 3 NBA players statistically from any season in the past 10 years (minimum 50 Games Played)
1. A player who averaged 20+ points that season
2. A player who averaed few than 10 points per game and 25 or more minutes per game that season
3. A player who averaged fewer than 10 minutes per game

For each player:
- Analyze their Usage and scoring effiency
- Provide 2-3 strengths and 2-3 weaknesses

---

## Introduction
This analysis examines player performance across varying roles and levels of offensive involvement during the 2024–2025 NBA season. The goal is to evaluate how players of different usage profiles contribute to their teams through both basic and advanced statistical metrics — identifying not just what they produce, but how efficiently they do so.

All data used in this report was gathered from NBA.com’s official statistics portal, which provides publicly available data through its player stats API. After obtaining the full dataset for the 2024–2025 season, the data was filtered to include only players meeting the assignment criteria:

1. One player averaging 20 or more points per game (primary scorer)
2. One player averaging fewer than 10 points per game but logging 25 or more minutes per game (high-minute, low-scoring contributor)
3. One player averaging fewer than 10 minutes per game (limited rotation or developmental player)

By analyzing these three player profiles, the report aims to demonstrate how usage rate, scoring efficiency, and play style manifest differently depending on a player’s offensive role and opportunity.

For each selected player, we’ll examine:
1. Usage Rate and Scoring Efficiency — understanding how frequently they are involved in offensive possessions and how effectively they convert opportunities.
2. Strengths and Weaknesses — identifying 2–3 areas of clear impact and 2–3 areas where statistical trends suggest room for improvement.

Through this comparison, we’ll uncover insights into how context, role, and efficiency metrics intersect to shape player value beyond traditional box-score averages.

In [2]:
# Import Libraries
import numpy as np
import pandas as pd
from datetime import datetime
from collections import Counter

today = datetime.today() # Get Today's Date
formatted_date = today.strftime('%m%d%Y') # Strip Today's Date in the MMDDYYYY Format

The data collected will be all players from the 2025-2026 NBA season that fit the criteria for the assignment.

In [4]:
# Load the CSV file into a DataFrame
player_data = pd.read_excel('Player-Data-Box-Score-Averages-2024-2025.xlsx')
team_data = pd.read_excel('Team-Data-Box-Score-Averages-2024-2025.xlsx')

### Usage
Usage is *the percentage of the team's possessions a player finishes while on the court*. The purpose of this metric is to measure how involved in finishing possessions a player is.

### Effective Field Goal Percentage (eFG%)
The Effective Field Goal percentage is a *player's shooting effiency that **adjusts** for the fact that a made 3PT shot is worth **1.5x** a made 2PT shot*. The purpose of this metric is to improve upon **FG%** as a measure for shooting.

### True Shooting Percentage (TS%)
The True Shooting percentage is a *player's shooting effiency taking into account the value of made 2s, 3s, and Free Throws*. The purpose of this metric is to measure a player's **scoring effiency**. 

**Note**: It's important to remember the difference between a *Field Goal Attempt (FGA)* and a *Shot Attempt*. A FGA is *any shot where a player is not fouled or makes the attempt*. A Shot Attempt is *any shot a player attempts, including if they were fouled and missess*.

In [6]:
# Define Estimated Advanced Metrics

# Shot Attempts
def shot_attempts(fga, fta):
    shot_attempts = fga + 0.44*fta
    return shot_attempts

#########################
# Usage
def usage (fga, fta, tov, tm_mp, mp, tm_fga, tm_fta, tm_tov, gp, tm_gp):
    # If any data is missing, then we return NAN
    if any(pd.isna(x) for x in [fga, fta, tov, tm_mp, mp, tm_fga, tm_fta, tm_tov]):
        return np.nan
    
    num = (fga + 0.44 * fta + tov) * (tm_mp)
    denom = (mp * (tm_fga + 0.44 * tm_fta + tm_tov))
    
    # Check for Divide-by-Zero
    if denom == 0:
        return np.nan

    # Calculate Usage
    usage = num / denom
    return usage
    
#########################
# Effective Field Goal Percentage (eFG%)
def eFG_percentage (fgm, made_3s, fga):
    num = (fgm + 0.5 * made_3s)
    denom = fga

    # Check for Divide-by-Zero
    if fga == 0:
        return np.nan
    
    # Calculate Effective Field Goal Percentage
    efg = num / denom
    return efg

#########################
# True Shooting Percentage (TS%)
def TS_percentage (pts, fga, fta):
    num = pts
    denom = 2 * (fga + 0.44 * fta)

    # Check for Divide-by-Zero
    if denom == 0:
        return np.nan

    # Calculate True Shooting Percentage
    ts = num / denom
    return ts

#########################
# Per 36-Minute Stat Calculator
def per36(stat_per_game, minutes_per_game):
    
    return stat_per_game * (36 / minutes_per_game)

#########################
# Gather Player Statistics
def get_player_data(df, player_name):
    player_df = df[df['Player'] == player_name]
    
    return player_df.reset_index(drop = True)

# Gather Team Statistics
def get_team_data(df, team_name):    
    team_df = df[df['Team'] == team_name]
    return team_df.reset_index(drop = True)

#########################
# Define Player Analysis
def calculate_player_metrics(player_df, team_df):
    # Calculate the Player's Shot Attempts
    player_shot_attempts = shot_attempts(fga = player_df["FGA"][0],
                                         fta = player_df["FTA"][0]
                                        )

    # Calculate the Player's Usage
    player_usage = usage(fga = player_df["FGA"][0], 
                         fta = player_df["FTA"][0], 
                         tov = player_df["TOV"][0],
                         tm_mp = team_df["Min"][0],
                         mp = player_df["Min"][0],
                         tm_fga = team_df["FGA"][0],
                         tm_fta = team_df["FTA"][0],
                         tm_tov = team_df["TOV"][0],
                         gp = player_df["GP"][0],
                         tm_gp = team_df["GP"][0]
                        )
    
    # Calculate the Effective Field Goal Percentage (eFG%)
    player_efg = eFG_percentage(player_df["FGM"][0],
                                player_df["3PM"][0],
                                player_df["FGA"][0]
                               )

    # Calculate the True Shooting Percentage (TS%)
    player_ts = TS_percentage(player_df["PTS"][0],
                              player_df["FGA"][0],
                              player_df["FTA"][0]
                             )

    # Calculate the PTS per Game at Per 36
    pts36 = per36(player_df["PTS"][0],
                  player_df["Min"][0]
                 )
    
    print("Player Metrics for " + player_df["Player"][0])
    print("Team: " + team_df["Team"][0])
    print("GP: " + str(player_df["GP"][0]))
    print("Minutes Per Game (MPG): " + str(player_df["Min"][0]))
    print("Points Per Game (PPG): " + str(player_df["PTS"][0]))
    print("Shot Attempts: " + str(round(player_shot_attempts, 2)))
    print("Usage Rate: " + str(round(player_usage*100, 2)) + "%")
    print("eFG%: " + str(round(player_efg*100, 2)) + "%")
    print("True Shooting Percentage: " + str(round(player_ts*100, 2)) + "%")
    print("PTS per 36: " + str(round(pts36, 2)) + " PTS")
    
    return [player_shot_attempts, player_usage, player_efg, player_ts]

In [7]:
# Clean the Player Data to include who is eligible for each Question
player_data["Question Type"] = None

# Mark Players that are eligible for Question 1, Question 2, and Question 3
player_data.loc[player_data["PTS"] >= 20, "Question Type"] = "Q1"
player_data.loc[(player_data["PTS"] <= 10) & (player_data["Min"] >= 25), "Question Type"] = "Q2"
player_data.loc[player_data["Min"] <= 10, "Question Type"] = "Q3"

### Question 1
Program to Calculate the Player Data for Each Eligible Player for Q1

```
q1 = player_data[player_data["Question Type"] == "Q1"]
all_ts = []

for player in player_data[player_data["Question Type"] == "Q1"]["Player"]:
    player_metrics = get_player_data(player_data, player)
    team_metrics = get_team_data(team_data,
                                 player_data.loc[player_data["Player"] == player, "Team"].values[0]
                                )
    analysis = calculate_player_metrics(player_metrics, team_metrics)
    all_ts.append(analysis[-1])
    print("")

import statistics as s
s.mean(all_ts)
```

### Question 2
Program to Calculate the Player Data for Each Eligible Player for Q2

```
q1 = player_data[player_data["Question Type"] == "Q2"]
all_ts = []

for player in player_data[player_data["Question Type"] == "Q1"]["Player"]:
    player_metrics = get_player_data(player_data, player)
    team_metrics = get_team_data(team_data,
                                 player_data.loc[player_data["Player"] == player, "Team"].values[0]
                                )
    analysis = calculate_player_metrics(player_metrics, team_metrics)
    all_ts.append(analysis[-1])
    print("")

import statistics as s
s.mean(all_ts)
```

### Question 3
Program to Calculate the Player Data for Each Eligible Player for Q3

```
q1 = player_data[player_data["Question Type"] == "Q3"]
all_ts = []

for player in player_data[player_data["Question Type"] == "Q1"]["Player"]:
    player_metrics = get_player_data(player_data, player)
    team_metrics = get_team_data(team_data,
                                 player_data.loc[player_data["Player"] == player, "Team"].values[0]
                                )
    analysis = calculate_player_metrics(player_metrics, team_metrics)
    all_ts.append(analysis[-1])
    print("")

import statistics as s
s.mean(all_ts)
```

# PLAYER ANALYSIS FOR QUESTION 1

## Cade Cunningham
- Player Name: Cade Cunningham
- Season: 2024-2025
- Team: Detroit Pistons

Key Stats:
- Games Played (GP): 70
- Minutes Per Game (MPG): 35.0
- Points Per Game (PPG): 26.1

Usage & Scoring Efficiency Analysis

- Shot Attempts: 23.13
- Usage Rate: 33.18%
- Effective Field Goal: 52.16%
- True Shooting: 56.42%
-----
### Strengths & Weaknesses
Cunningham’s 33.18 % usage rate places him among the league’s highest, indicating that roughly one of every three team possessions ends with him taking a shot, drawing a foul, or committing a turnover. Despite that workload, his 52.16 % eFG % and 56.42 % TS % remains very good, reflecting solid efficiency under heavy defensive pressure on the Detroit Pistons.

### Strengths

Strength 1 - High Offensive Responsibility and Solid Efficiency: Cade Cunningham has a high-offensive responsibility and has solid efficiency. Cunningham averages at about a 33% Usage Rate, and maintains a 56% TS%. This places him at a Top-Ten % of NBA Players in terms of usage, and his TS% puts him in player comparisons between Luka Doncic and Jayson Tatum. 

Strength 2 - Three-Level Scoring: Cade Cunningham averages about 23 Shot Attempts per game. Looking at the data from NBA.com, Cunningham has a wide variety of shots from the field, and he goes to the line about 5 times a game. He is deadly from inside the paint shooting 53.9% from less-than-8ft away from the basket. 

Strength 3 - Durability and Conditioning: Cunningham played 70 games at 35 MPG, which shows his excellent stamina and consistency. The coaches and team can rely on him for sustained minutes and late-game creation. This is crucial for a player with a high Usage rate of 33%. 


### Weaknesses

Weakness 1 - Efficiency vs Volume Tradeoff: Although 56.4% TS% is respectable, it is still below average for players that qualify for this question, which is 59.1% TS%. 

Weakness 2 - Reliance on Difficult Shot Creation: The high Usage Rate of 33% often forces Cunningham into self-created attempts late in the shot clock. His 52% eFG% suggests he could benefit from more catch-and-shoot or assisted looks. Last season, Cunningham was assisted by Jalen Duren the most with 55 assists. With Jaden Ivey returning to the Pistons as a secondary creator, this could help Cunningham’s game by lowering his Usage Rate and increasing his eFG%.

Weakness 3 (Optional) - Offensive Turnover Pressure: High-usage guards naturally generate more turnovers, and Cade Cunningham is no exception. Cunningham averages 4.4 TOVs per game, which is the second-highest in the league. Reducing live-ball turnovers and improving decision-speed and decision-making could elevate his efficiency even further.

In [10]:
# Question 1
# Cade Cunningham
cade_cunningham = get_player_data(player_data, "Cade Cunningham")
detroit_pistons = get_team_data(team_data, "DET")
cade_metrics = calculate_player_metrics(cade_cunningham, detroit_pistons)

Player Metrics for Cade Cunningham
Team: DET
GP: 70
Minutes Per Game (MPG): 35.0
Points Per Game (PPG): 26.1
Shot Attempts: 23.13
Usage Rate: 33.18%
eFG%: 52.16%
True Shooting Percentage: 56.42%
PTS per 36: 26.85 PTS


# PLAYER ANALYSIS FOR QUESTION 2

## Bub Carrington
- Player Name: Bub Carrington
- Season: 2024-2025
- Team: Washington Wizards

Key Stats:
- Games Played (GP): 82
- Minutes Per Game (MPG): 30.0
- Points Per Game (PPG): 9.8

Usage & Scoring Efficiency Analysis

- Shot Attempts: 9.8
- Usage Rate: 15.73%
- Effective Field Goal: 50.0%
- True Shooting: 51.36%
- Per 36 Minutes: 11.8 PTS


### Strengths & Weaknesses
Bub Carrington was a rookie guard for the 2024-2025 Washington Wizards, and he was a consistent rotational contributor who logged heavy minutes across a full, 82-game season. Though his scoring volume was modest, his availability, discipline, and role execution made him a reliable fixture in Washington’s backcourt rotation.

### Strengths

Strength 1 - Consistency and Availability: Bub Carrington appeared in all 82 games, and he averaged 30 minutes per night. That consistency allowed him to build rhythm and gain valuable in-game experience, especially as an NBA rookie.

Strength 2 - Role Discipline and Complementary Play: Carrington’s Usage Rate of 15.7% shows he doesn’t force the offense, and he stays within the team structure. He contributes through off-ball movement, secondary playmaking (4.4 ASTs), and spacing (33.9% 3P% - around league average for rookies), for higher-usage players like Jordan Poole.

Strength 3 - Foundational Shooting Mechanics and Confidence: Despite the modes 50% eFG%, Carrington displayed a willingness to play in a limited-role, and he made the most of his chances with open looks and a balanced shot profile. He averaged 1.7 3PMs per game, which suggests a solid foundation for efficiency and growth as he continues to gain experience in the league.

### Weaknesses

Weakness 1 - Limited Scoring Efficiency: Carrington’s 51.36% TS% lags behind the league average of 57%. This signals that he struggles to convert possessions into efficient points. Improving finishing at the rim, drawing more fouls, and improving his 3PM% could raise his scoring impact and TS%. 

Weakness 2 - Low Offensive Aggression: A Usage Rate under 16% indicates a lack of assertiveness in creating shots. Again, the role assigned to him didn’t require high volume, he could attack more off-the-drible or in pick-and-roll actions to make himself less predictable and add value to the offense. An increased role with Jordan Poole leaving this off-season could help with Carrington gaining that confidence and comfortability in that aspect.

Weakness 3 - Developing Rim Pressure and Physicality: Carrington relies heavily on jumpers; limited rim attempts and free-throw trips constrain his efficiency ceiling. Carington made 55-out-of-110 FGAs less-than-5ft from-the-basket, and he made 99-295 FGAs between 25-29 ft away from-the-basket. In addition, he averaged 1 FTA per game. Introducing a more varied shot diet could help in his efficiency, and getting to the basket more can help get him to the line more.


In [12]:
# Question 2
# Bub Carrington
bub_carrington = get_player_data(player_data, "Bub Carrington")
washington_wizards = get_team_data(team_data, "WAS")
bub_metrics = calculate_player_metrics(bub_carrington, washington_wizards)

Player Metrics for Bub Carrington
Team: WAS
GP: 82
Minutes Per Game (MPG): 30.0
Points Per Game (PPG): 9.8
Shot Attempts: 9.54
Usage Rate: 15.73%
eFG%: 50.0%
True Shooting Percentage: 51.36%
PTS per 36: 11.76 PTS


# PLAYER ANALYSIS FOR QUESTION 3

## Johnny Furphy

- Player Name: Johnny Furphy
- Season: 2024-2025
- Team: Indiana Pacers
Key Stats:
- Games Played (GP): 50
- Minutes Per Game (MPG): 7.6
- Points Per Game (PPG): 2.1

Usage & Scoring Efficiency Analysis

- Shot Attempts: 1.98
- Usage Rate: 12.37%
- Effective Field Goal: 47.37%
- True Shooting: 53.14%
- Per 36: 9.95 PTS


### Strengths & Weaknesses
As a low-minute rookie for the Indiana Pacers, Johnny Furphy filled a specialist-type role, focused on spacing-the-floor and providing situational shooting off-the-bench. His small sample size limits statistical stability, but the metrics offer an early look at his offensive profile and developmental trajectory. It's important to note that his Per 36 estimates indicate that given a larger role, he could be averaging close to 10 PTS per game, which aligns closely with other low-usage role players who provide spacing.

### Strengths

Strength 1 - Floor Spacing and Shooting Confidence: Johnny Furphy’s offensive role is focused on perimeter spacing as he averaged 30% from 3P%, 1 3PA, and 1.8 FGAs per game - meaning half of his shots were from downtown. With his limited minutes, he showed his willingness to accept a role, take open threes, and help the team win. He embodied sacrifice. His 47% eFG% is below average, but it is not alarming or detrimental given his specific role and small sample of attempts 

Strength 2 - Low-Mistake Role Player: A 12% Usage Rate and minimal TOVs (0.2 TOVs per game) indicate Furphy rarely forces the offense; rather, he flows with the offense. He stays within the system, making simple reads and contributing to the Pacers’ ball movement. 

Strength 3 - Developmental Potential as a Specialist: Note that Furphy is just 20-years-old, and he is coming into his second-year in the NBA. He will be returning to an Indiana Pacers team with low-expectations due to injuries to their star guard, Tyrese Haliburton, and off-season losses in Myles Turner. This could pose an increased role for Furphy as he displays foundational tools in length, shooting touch, and positional size. 


### Weaknesses

Weakness 1 - Limited Offensive Impact: Furphy only averages 1.98 Shot Attempts per game. Increasing activity through cuts, off-ball screens, or offensive rebounds would make him a more dynamic threat in low-minute stints.

Weakness 2 - Below-Average Shooting Efficiency: A 47% eFG% and 53% TS% are below league benchmarks for shooting specialists. Furphy will need to improve his catch-and-shoot consistency and learn to adjust to NBA close-outs. These will be key to solidifying his identity as a floor spacer.

Weakness 3 - Defensive Physicality and Strength: Furphy needs to improve his on-ball defense (0.4 STLs and 0.2 BLKs) and rebounding (1.4 REBs) to stay on-the-floor. Greater strength and awareness will translate his athletic tools and create consistent rotation minutes for himself.

In [14]:
# Question 3
# Johnny Furphy
johnny_furphy = get_player_data(player_data, "Johnny Furphy")
indiana_pacers = get_team_data(team_data, "IND")
johnny_metrics = calculate_player_metrics(johnny_furphy, indiana_pacers)

Player Metrics for Johnny Furphy
Team: IND
GP: 50
Minutes Per Game (MPG): 7.6
Points Per Game (PPG): 2.1
Shot Attempts: 1.98
Usage Rate: 12.37%
eFG%: 47.22%
True Shooting Percentage: 53.14%
PTS per 36: 9.95 PTS
